# Handcrafted Fully Connected Neural Net (FCNN)

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [3]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

### Standardizing the Data 

In [4]:
# The model was tested with both standardization, and normalization. Normalization doesn't seem to 
# give good results, that's why we finalized our design with standardization. 
# standardize the training data
from sklearn.preprocessing import StandardScaler
scale= StandardScaler()

Xtr_standardized = scale.fit_transform(Xtr) # revise this line as needed
Xts_standardized = scale.fit_transform(Xts) # revise this line as needed
ytr_standardized = ytr # revise this line as needed

Xtr_torch = torch.Tensor(Xtr_standardized)
Xts_torch = torch.Tensor(Xts_standardized)
ytr_torch = torch.Tensor(ytr_standardized)

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr_standardized, delimiter=",")

### Train Test Split, and Creation of Pytorch Dataloaders 

In [5]:
x_train,x_test,y_train,y_test = train_test_split(Xtr_torch,ytr_torch,test_size=0.2,random_state = random.randint(0,1000))
train_ds = TensorDataset(x_train, y_train)
test_ds =  TensorDataset(x_test, y_test)
batch_size = 32
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)


### Handcrafted Model 

In [49]:
# create a model
d_in = Xtr.shape[1]
d_out = 1
nin = 8
nh=100
nout=1

activation_fn = nn.Sigmoid()
dropout_prob = 0.5

class Net(nn.Module):
    def __init__(self,nin,nh,nout):
        super(Net,self).__init__()
        self.sigmoid = activation_fn
        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.batch_normal = nn.BatchNorm1d(95)
        self.dropout = nn.Dropout(dropout_prob)
        self.Dense1 = nn.Linear(nin,95)
        self.Dense2 = nn.Linear(95,95)
        self.Dense3 = nn.Linear(95,80)
        self.Dense4 = nn.Linear(80,nout)
        
    def forward(self,x):
        x = self.sigmoid(self.Dense1(x))
        x = self.dropout(x)
#         x = self.batch_normal(x)
        x = self.sigmoid(self.Dense2(x))
#         x = self.dropout(x)
#         x = self.batch_normal(x)
        x = self.sigmoid(self.Dense3(x))
#         x = self.dropout(x)
#         x = self.batch_normal(x)
        out = self.sigmoid(self.Dense4(x))
        return out

model1 = Net(nin=nin, nh=nh, nout=nout)

print(str(model1))

# Usually, we would train the model at this point. 
# But this is only a demo, so we'll use the randomly initialized weights.
# num_epoch = 40
lrate = 2e-3
# decay = lrate/num_epoch

opt = optim.Adam(model1.parameters(), lr=lrate)

# lambda1 = lambda epoch: (1-decay)*epoch
# scheduler = optim.lr_scheduler.LambdaLR(opt, lr_lambda=lambda1)

criterion = nn.BCELoss()

Net(
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=None)
  (relu): ReLU()
  (tanh): Tanh()
  (batch_normal): BatchNorm1d(95, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (Dense1): Linear(in_features=8, out_features=95, bias=True)
  (Dense2): Linear(in_features=95, out_features=95, bias=True)
  (Dense3): Linear(in_features=95, out_features=80, bias=True)
  (Dense4): Linear(in_features=80, out_features=1, bias=True)
)


### Training the model 

In [50]:
# TODO
import time

num_epoch = 100

a_tr_loss = np.zeros([num_epoch])
a_tr_accuracy = np.zeros([num_epoch])
a_ts_loss = np.zeros([num_epoch])
a_ts_accuracy = np.zeros([num_epoch])

for epoch in range(num_epoch):
    correct = 0 # initialize error counter
    total = 0 # initialize total counter
    batch_loss = []
    tic = time.time()
    
    model1.train() # put model in training mode
    # iterate over training set
    for train_iter, data in enumerate(train_dl):
        x_batch,y_batch = data
        y_batch = y_batch.view(-1,1)
        out = model1(x_batch)
        # Compute Loss
        loss = criterion(out.clamp(0,1),y_batch.type(torch.float))
        batch_loss.append(loss.item())
        # Zero gradients
        opt.zero_grad()
        # Compute gradients using back propagation
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        predicted = out.clamp(0,1).round().type(torch.long)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
    a_tr_loss[epoch] = np.mean(batch_loss) # Compute average loss over epoch
    a_tr_accuracy[epoch] = 100*correct/total
    
    correct = 0
    total = 0
    batch_loss = []
    model1.eval() # put model in evaluation mode
    with torch.no_grad():
        for data in test_dl:
            images, labels = data
            labels = labels.view(-1,1)
            out = model1(images)
            batch_loss.append(criterion(out.clamp(0,1),labels.type(torch.float)).item())
            predicted = out.clamp(0,1).round().type(torch.long)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    a_ts_loss[epoch] = np.mean(batch_loss)
    a_ts_accuracy[epoch] = 100*correct/total
    elapsed = time.time()-tic
    
    # Print details every print_mod epoch
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
          +'Train Accuracy: {0:.2f}    Test Loss: {1:.3f}   '.format(a_tr_accuracy[epoch], a_ts_loss[epoch])
          +'Test Accuracy: {0:.2f}   '.format(a_ts_accuracy[epoch])
          +'Time: {0:.2f}sec'.format(elapsed))

Epoch:  1   Train Loss: 0.549   Train Accuracy: 73.15    Test Loss: 0.476   Test Accuracy: 78.05   Time: 0.51sec
Epoch:  2   Train Loss: 0.484   Train Accuracy: 78.74    Test Loss: 0.474   Test Accuracy: 78.35   Time: 0.62sec
Epoch:  3   Train Loss: 0.477   Train Accuracy: 78.85    Test Loss: 0.468   Test Accuracy: 78.05   Time: 0.59sec
Epoch:  4   Train Loss: 0.471   Train Accuracy: 78.85    Test Loss: 0.456   Test Accuracy: 78.55   Time: 0.60sec
Epoch:  5   Train Loss: 0.457   Train Accuracy: 79.11    Test Loss: 0.442   Test Accuracy: 78.70   Time: 0.52sec
Epoch:  6   Train Loss: 0.447   Train Accuracy: 79.83    Test Loss: 0.435   Test Accuracy: 79.60   Time: 0.64sec
Epoch:  7   Train Loss: 0.441   Train Accuracy: 79.97    Test Loss: 0.428   Test Accuracy: 80.25   Time: 0.65sec
Epoch:  8   Train Loss: 0.440   Train Accuracy: 80.31    Test Loss: 0.425   Test Accuracy: 80.65   Time: 0.54sec
Epoch:  9   Train Loss: 0.438   Train Accuracy: 80.29    Test Loss: 0.421   Test Accuracy: 80.90

Epoch: 74   Train Loss: 0.347   Train Accuracy: 85.10    Test Loss: 0.332   Test Accuracy: 86.90   Time: 0.66sec
Epoch: 75   Train Loss: 0.351   Train Accuracy: 85.00    Test Loss: 0.336   Test Accuracy: 86.35   Time: 0.54sec
Epoch: 76   Train Loss: 0.349   Train Accuracy: 85.40    Test Loss: 0.331   Test Accuracy: 86.50   Time: 0.68sec
Epoch: 77   Train Loss: 0.347   Train Accuracy: 85.39    Test Loss: 0.327   Test Accuracy: 86.95   Time: 0.80sec
Epoch: 78   Train Loss: 0.350   Train Accuracy: 84.85    Test Loss: 0.335   Test Accuracy: 86.95   Time: 0.56sec
Epoch: 79   Train Loss: 0.345   Train Accuracy: 85.49    Test Loss: 0.330   Test Accuracy: 87.10   Time: 0.60sec
Epoch: 80   Train Loss: 0.348   Train Accuracy: 84.96    Test Loss: 0.330   Test Accuracy: 86.80   Time: 0.71sec
Epoch: 81   Train Loss: 0.349   Train Accuracy: 84.85    Test Loss: 0.331   Test Accuracy: 87.35   Time: 0.65sec
Epoch: 82   Train Loss: 0.348   Train Accuracy: 85.05    Test Loss: 0.329   Test Accuracy: 87.30

In [51]:
# compute the training accuracy
with torch.no_grad():
    predict = model1(torch.Tensor(Xtr_standardized)).detach().numpy().ravel()
    

auc = roc_auc_score(ytr_standardized,predict)
print('training auc: ',auc)

training auc:  0.9199856429850293


### Saving the model 

In [52]:
# save the model: you must use the .pth format for pytorch models!
model_savepath = 'model1.pth'

# To save a PyTorch model, we first pass an input through the model, 
# and then save the "trace". 
# For this purpose, we can use any input. 
# We will create a random input with the proper dimension.
x = torch.randn(d_in) # random input
x = x[None,:] # add singleton batch index
with torch.no_grad():
    traced_cell = torch.jit.trace(model1, (x))

# Now we save the trace
torch.jit.save(traced_cell, model_savepath)

In [53]:
# generate kaggle submission file using the validation script
!python {"validation.py " + model_savepath + " --Xts_path " + Xts_savepath + " --Xtr_path " + Xtr_savepath + " --yts_hat_path " + yts_hat_savepath } 

training auc =  0.9199856429850293
test label confidences saved in yts_hat_pytorch.csv
